# Enunciado

Parte 2: Funções Avançadas
Uso de UDFs (User Defined Functions)
Crie uma função em Python que converte idades para categorias:
Menor que 30: "Jovem"
Entre 30 e 40: "Adulto"
Maior que 40: "Senior"
Aplique essa função ao DataFrame usando uma UDF.
Funções de Janela
Use funções de janela para adicionar uma coluna que mostre a diferença de idade entre cada indivíduo e a média de idade do seu "Occupation".

In [5]:
# Criação do DataFrame e importações dos modulos
from pyspark.sql import SparkSession
from pyspark.sql.functions import avg, col
from pyspark.sql.window import Window
from pyspark.sql.functions import mean

# Inicializando a SparkSession
spark = SparkSession.builder.appName("Data Engineer Test").getOrCreate()

# Inicialização das variaveis
data = [
    ("Alice", 34, "Data Scientist"),
    ("Bob", 45, "Data Engineer"),
    ("Cathy", 29, "Data Analyst"),
    ("David", 35, "Data Scientist")
]
columns = ["Name", "Age", "Occupation"]

# Criando o DataFrame
df = spark.createDataFrame(data, schema=columns)

df.show()


+-----+---+--------------+
| Name|Age|    Occupation|
+-----+---+--------------+
|Alice| 34|Data Scientist|
|  Bob| 45| Data Engineer|
|Cathy| 29|  Data Analyst|
|David| 35|Data Scientist|
+-----+---+--------------+



In [6]:
# Criação da função
def convert_age_to_category(age):
    # Define a categoria com base na idade
    if age < 30:
        category = 'Jovem'
    elif  age >= 30 or age <= 40:
        category = 'Adulto'
    elif age > 40:
        category = 'Senior'
    
    return category

print(convert_age_to_category(30))

Adulto


In [7]:
# Utilizando a UDF com o Spark
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType

# Registrando a função como UDF
age_category_udf = udf(convert_age_to_category, StringType())

# Aplicando a UDF ao DataFrame
df_with_category = df.withColumn("Age Category", age_category_udf(col("Age")))


df_with_category.show()

+-----+---+--------------+------------+
| Name|Age|    Occupation|Age Category|
+-----+---+--------------+------------+
|Alice| 34|Data Scientist|      Adulto|
|  Bob| 45| Data Engineer|      Adulto|
|Cathy| 29|  Data Analyst|       Jovem|
|David| 35|Data Scientist|      Adulto|
+-----+---+--------------+------------+



In [9]:
# Utilizando as Funções de Janela

# Definindo a janela de partição por 'Occupation'
window = Window.partitionBy("Occupation")

# Calculando a média de idade por ocupação e adicionando como nova coluna
df_with_mean_age = df.withColumn("Mean Age", mean(col("Age")).over(window))

# Calculando a diferença entre a idade de cada indivíduo e a média de idade da sua ocupação
df_with_age_difference = df_with_mean_age.withColumn("Age Difference", col("Age") - col("Mean Age"))


df_with_age_difference.show()

+-----+---+--------------+--------+--------------+
| Name|Age|    Occupation|Mean Age|Age Difference|
+-----+---+--------------+--------+--------------+
|Cathy| 29|  Data Analyst|    29.0|           0.0|
|  Bob| 45| Data Engineer|    45.0|           0.0|
|Alice| 34|Data Scientist|    34.5|          -0.5|
|David| 35|Data Scientist|    34.5|           0.5|
+-----+---+--------------+--------+--------------+

